In [1]:
import re
import cv2
import numpy as np
from PIL import Image
from tqdm.auto import tqdm
from pyzbar.pyzbar import *

In [193]:
NUM = 12
cap = cv2.VideoCapture(f"C:/QRcodePy/2021년12월/{NUM}.MOV")    # 파일 입력

# cap = cv2.VideoCapture(0)    # 첫번째 카메라
# cap = cv2.VideoCapture("rtsp://ID:PW@ip-address")    # IP카메라

In [194]:
# 카메라 : 카메라가 없어도 True일 수 있음 ==> read()에서 False
# 파일 : 파일을 읽을 수 없으면 False
cap.isOpened()

True

In [195]:
# 속성 받아오기
# cap.get(cv2.CAP_PROP_FRAME_WIDTH) = 가로 해상도(3)
# cap.get(cv2.CAP_PROP_FRAME_HEIGHT) = 세로 해상도(4)
# cap.get(cv2.CAP_PROP_FRAME_COUNT) = 총 프레임 수
# cap.get(cv2.CAP_PROP_FPS) = 프레임율(5)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)

print('Frame width:', width)
print('Frame height:', height)
print('Frame count:', frame_count)
print('FPS:', fps)

Frame width: 3840
Frame height: 2160
Frame count: 179
FPS: 59.96649916247906


In [196]:
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
# cap.set(cv2.CAP_PROP_FPS, 30)
cap.get(cv2.CAP_PROP_FRAME_COUNT)

179.0

In [197]:
# 원하는 QR만 전처리

# <QR Format>
# 식당: NWTAVR-MA-202111-식당이름
# 직원: 2021-11-4077-01              ==> 다음달부터 NWTAVR-MS-202112-4077-01

# HEADER = 노원구청 교통행정과 등록팀 (내 QR인지 식별자 역할)
# CLASS : 가맹점 = Member of Affiliation / 직원 = Member of Staff
QR_HEADER = "NWTAVR"                  # 6
QR_CLASS_MA = "MA"                    # 2
QR_CLASS_MS = "MS"                    # 2
QR_YEAR = "2021"                      # 4
QR_MONTH = "12"                       # 2
QR_YEAR_MONTH = QR_YEAR + QR_MONTH    # 6
QR_SN = ["0"+str(i) for i in range(1,10)] + list(map(str, range(10, 21)))    # ["01", "02", ... "19", "20"]

affil_list = ["노란코끼리", "어장촌생선구이", "횡성목장", "항도(港都)", "전주콩나루", "옛날칼국수", "칠리사이공", "도나한우", "북경(北京)", "명문식당", "일성스시", "새싹비빔밥", "구내매점"]

code_list =  ["4080",    "4076",    "4079",    "4069",    "4078",    "4071",    "4068",    "4075",    "4077"   ]
name_list =  ["전현호",  "정성욱",  "박선녕",  "신상용",  "이정선",  "이희영",  "최미연",  "이재영",  "김재형" ]
color_list = ["#E32636", "#FFA500", "#FF5511", "#0FF1C3", "#AB3ED8", "#AAEE00", "#1155FF", "#FCE205", "#119617"]

name_by_code =  {"4080":"전현호",  "4076":"정성욱",  "4079":"박선녕",  "4069":"신상용",  "4078":"이정선",  "4071":"이희영",  "4068":"최미연",  "4075":"이재영",  "4077":"김재형"}
color_by_code = {"4080":"#E32636", "4076":"#FFA500", "4079":"#FF5511", "4069":"#0FF1C3", "4078":"#AB3ED8", "4071":"#AAEE00", "4068":"#1155FF", "4075":"#FCE205", "4077":"#119617"}

MA_pattern = re.compile(QR_HEADER + "-" + QR_CLASS_MA + "-" + QR_YEAR_MONTH + "-" + ".+")    # NWTAVR-MA-202111-노란코끼리
# MS_pattern = re.compile(QR_YEAR + "-" + QR_MONTH + "-" + "[0-9]{4}-[0-9]{2}")                # 2021-11-4077-01
MS_pattern = re.compile(QR_HEADER + "-" + QR_CLASS_MS + "-" + QR_YEAR_MONTH + "-" + ".+")  # NWTAVR-MS-202112-4077-01 (12월부터)


In [198]:
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter(f"C:/QRcodePy/2021년12월/QR_result_{NUM}.avi", fourcc, fps, (width, height))

In [199]:
detected_MA_list = []    # 식당이름
detected_MS_list = []    # 식권QR

# cv2.namedWindow("Frame", cv2.WINDOW_NORMAL)

for frame_index in tqdm(range(frame_count)):

	# 읽어오지 못한 경우 이 시점에서 success = False
	success, frame = cap.read()
	if success == False:
		break
	
	# 프레임에서 QR code 찾기
	result = decode(frame, symbols=[ZBarSymbol.QRCODE])
	num_of_QR = len(result)
	# print(f"[전처리 전] 발견된 QR개수: {num_of_QR}")
    
    
	# 만약 하나도 발견되지 않았다면 다음 Frame
	if num_of_QR == 0:
		print("QR code not detected")
		continue
	
	
	# 프레임 1개에서 발견한 여러개의 QR
	for eachQR in result:
	
		# QR 형식검증 : MS/MA패턴
		eachQR_data = eachQR.data.decode('utf-8')
		match_result_MA = MA_pattern.match(eachQR_data)
		match_result_MS = MS_pattern.match(eachQR_data)

		if match_result_MA is None:
			if match_result_MS is None:
				result.remove(eachQR)
				continue
			else:
				detected_MS_list.append(match_result_MS.group())
		else:
			detected_MA_list.append(match_result_MA.group())

		# QR 테두리 선그리기
		points = len(eachQR.polygon)     # QR 1개당 인식된 폴리곤 테두리 점의 개수(보통 4각형=4개)

		# 경계선 컬러 설정
		if match_result_MS is not None:
			_header, _ms, _yyyymm, code, sn = eachQR.data.decode('utf-8').split('-')
			color_hex_selected = color_by_code[code]
			color_selected = tuple(int(color_hex_selected[i:i+2], 16) for i in (5, 3, 1))
		else:
			header, classifier, year_month, affil_name = eachQR.data.decode('utf-8').split('-')
			color_selected = (0, 0, 255)    # 식당QR은 빨간색

		# QR 1개당 경계선 그리기
		for point in range(points):
			next_point = (point+1) % points
			cv2.line(frame, tuple(eachQR.polygon[point]), tuple(eachQR.polygon[next_point]), color_selected, 5)    # (B,G,R), 굵기
            
		# cv2.imshow("Frame", frame)    # 화면에 창 띄우기
	
	num_of_QR_preprocessed = len(result)
	# print("Frame  #{:3}/{:3} 처리 끝    ====>    전처리 전: {:2} \t 전처리 후: {:2}".format(frame_index+1, frame_count, num_of_QR, num_of_QR_preprocessed))
	out.write(frame)

# cv2.waitKey(0)                # 키보드에 키를 누를 때까지 기다림
# cv2.destroyAllWindows()       # 창 종료    
cap.release()
out.release()

  0%|          | 0/179 [00:00<?, ?it/s]

In [200]:
# QR 검출된 식당 개수(중복 포함)

len(detected_MA_list)

179

In [201]:
# 중복 제거한 "식당" ====> 1개가 정상

detected_MA_set = set(detected_MA_list)
MA_list = list(detected_MA_set)

print(len(MA_list))
print(MA_list)

1
['NWTAVR-MA-202112-새싹비빔밥']


In [202]:
# QR 검출된 직원 명수(중복 포함)

len(detected_MS_list)

179

In [203]:
# 중복 제거한 "직원" ====> 0개 이상

detected_MS_set = set(detected_MS_list)    # set은 자동 sort
MS_list = list(detected_MS_set)    # list는 자동 sort 안 됨
MS_list = sorted(MS_list)

print(len(MS_list))
print(MS_list)

1
['NWTAVR-MS-202112-4080-17']


In [204]:
# QR 검출된 직원 리스트
staff_list = []    # 이름 리스트
count_list = []    # 개수 리스트

if len(MS_list) != 0:
	_header, _ms, _yyyymm, current_code, _sn = MS_list[0].split("-")
	current_count = 0
	
	for index, eachMS in enumerate(MS_list):
		_header, _ms, _yyyymm, code, _sn = eachMS.split("-")
		
		if code != current_code:
			staff_list.append(current_code)
			count_list.append(current_count)
			
			current_code = code
			current_count = 1
		else:
			current_count += 1
		
		if (index == (len(MS_list) - 1)):
			staff_list.append(current_code)
			count_list.append(current_count)
			break
else:
	print("QR 검출된 직원이 없습니다.")


In [205]:
print(staff_list)
print(count_list)

['4080']
[1]


In [206]:
# python 3.7부터 dict = ordered
final_MS_dict = {}

# 전체 code_list : ['4080', '4076', '4079', '4069', '4078', '4071', '4068', '4075', '4077']
for _code in code_list:
	name = name_by_code[_code]
	
	if _code not in staff_list:
		final_MS_dict[name] = 0
	else:
		final_MS_dict[name] = count_list[staff_list.index(_code)]

final_MS_dict

{'전현호': 1,
 '정성욱': 0,
 '박선녕': 0,
 '신상용': 0,
 '이정선': 0,
 '이희영': 0,
 '최미연': 0,
 '이재영': 0,
 '김재형': 0}

In [207]:
final_date = MA_list[0].split("-")[2]
final_affil = MA_list[0].split("-")[3]

print(f"{final_date[:4]}.{final_date[4:]}.  {final_affil}  총{sum(final_MS_dict.values())}개")
print("="*32)

for key, val in zip(final_MS_dict.keys(), final_MS_dict.values()):
	print(f"{key} : {val:2}개")

2021.12.  새싹비빔밥  총1개
전현호 :  1개
정성욱 :  0개
박선녕 :  0개
신상용 :  0개
이정선 :  0개
이희영 :  0개
최미연 :  0개
이재영 :  0개
김재형 :  0개


In [208]:
if NUM == 1:
	final_MA_list = []
	final_MS_list = []

final_MA_list.append(final_affil)
final_MS_list.append(final_MS_dict)

In [209]:
print(final_MA_list)
print(final_MS_list)

['노란코끼리', '어장촌생선구이', '횡성목장', '항도(港都)', '전주콩나루', '옛날칼국수', '칠리사이공', '도나한우', '북경(北京)', '명문식당', '일성스시', '새싹비빔밥']
[{'전현호': 3, '정성욱': 1, '박선녕': 11, '신상용': 0, '이정선': 7, '이희영': 3, '최미연': 2, '이재영': 11, '김재형': 0}, {'전현호': 6, '정성욱': 0, '박선녕': 0, '신상용': 0, '이정선': 1, '이희영': 0, '최미연': 2, '이재영': 1, '김재형': 2}, {'전현호': 3, '정성욱': 4, '박선녕': 0, '신상용': 0, '이정선': 0, '이희영': 0, '최미연': 2, '이재영': 2, '김재형': 3}, {'전현호': 0, '정성욱': 1, '박선녕': 0, '신상용': 0, '이정선': 0, '이희영': 0, '최미연': 1, '이재영': 0, '김재형': 0}, {'전현호': 1, '정성욱': 2, '박선녕': 2, '신상용': 1, '이정선': 1, '이희영': 2, '최미연': 1, '이재영': 1, '김재형': 3}, {'전현호': 0, '정성욱': 1, '박선녕': 0, '신상용': 0, '이정선': 0, '이희영': 0, '최미연': 0, '이재영': 0, '김재형': 0}, {'전현호': 0, '정성욱': 0, '박선녕': 1, '신상용': 0, '이정선': 1, '이희영': 1, '최미연': 0, '이재영': 0, '김재형': 0}, {'전현호': 0, '정성욱': 6, '박선녕': 5, '신상용': 0, '이정선': 4, '이희영': 4, '최미연': 6, '이재영': 0, '김재형': 0}, {'전현호': 0, '정성욱': 1, '박선녕': 0, '신상용': 0, '이정선': 0, '이희영': 0, '최미연': 2, '이재영': 0, '김재형': 1}, {'전현호': 6, '정성욱': 4, '박선녕': 0, '신상용': 1, '이정선': 0, '이희영': 0,

## 동영상 파일 QR처리 끝

## Pandas 데이터프레임에 최종 입력 시작

In [210]:
import pandas as pd

column_list = ["직원성명", "노란코끼리", "어장촌생선구이", "횡성목장", "항도(港都)", "전주콩나루", "옛날칼국수", "칠리사이공", "도나한우", "북경(北京)", "명문식당", "일성스시", "새싹비빔밥", "구내매점"]
df = pd.DataFrame(columns=column_list)
df

,직원성명,노란코끼리,어장촌생선구이,횡성목장,항도(港都),전주콩나루,옛날칼국수,칠리사이공,도나한우,북경(北京),명문식당,일성스시,새싹비빔밥,구내매점


In [211]:
df['직원성명'] = name_list
for col_name, val_dict in zip(final_MA_list, final_MS_list):
	df[col_name] = val_dict.values()
df

,직원성명,노란코끼리,어장촌생선구이,횡성목장,항도(港都),전주콩나루,옛날칼국수,칠리사이공,도나한우,북경(北京),명문식당,일성스시,새싹비빔밥,구내매점
0,전현호,3,6,3,0,1,0,0,0,0,6,0,1,NaN
1,정성욱,1,0,4,1,2,1,0,6,1,4,0,0,NaN
2,박선녕,11,0,0,0,2,0,1,5,0,0,1,0,NaN
3,신상용,0,0,0,0,1,0,0,0,0,1,0,0,NaN
4,이정선,7,1,0,0,1,0,1,4,0,0,1,0,NaN
5,이희영,3,0,0,0,2,0,1,4,0,0,3,0,NaN
6,최미연,2,2,2,1,1,0,0,6,2,1,0,0,NaN
7,이재영,11,1,2,0,1,0,0,0,0,5,0,0,NaN
8,김재형,0,2,3,0,3,0,0,0,1,5,0,0,NaN


In [212]:
df['구내매점'] = [0, 0, 0, 0, 0, 0, 0, 0, 0]
df

,직원성명,노란코끼리,어장촌생선구이,횡성목장,항도(港都),전주콩나루,옛날칼국수,칠리사이공,도나한우,북경(北京),명문식당,일성스시,새싹비빔밥,구내매점
0,전현호,3,6,3,0,1,0,0,0,0,6,0,1,0
1,정성욱,1,0,4,1,2,1,0,6,1,4,0,0,0
2,박선녕,11,0,0,0,2,0,1,5,0,0,1,0,0
3,신상용,0,0,0,0,1,0,0,0,0,1,0,0,0
4,이정선,7,1,0,0,1,0,1,4,0,0,1,0,0
5,이희영,3,0,0,0,2,0,1,4,0,0,3,0,0
6,최미연,2,2,2,1,1,0,0,6,2,1,0,0,0
7,이재영,11,1,2,0,1,0,0,0,0,5,0,0,0
8,김재형,0,2,3,0,3,0,0,0,1,5,0,0,0


In [215]:
df2 = df.set_index('직원성명')
df2

,노란코끼리,어장촌생선구이,횡성목장,항도(港都),전주콩나루,옛날칼국수,칠리사이공,도나한우,북경(北京),명문식당,일성스시,새싹비빔밥,구내매점
직원성명,,,,,,,,,,,,,
전현호,3,6,3,0,1,0,0,0,0,6,0,1,0
정성욱,1,0,4,1,2,1,0,6,1,4,0,0,0
박선녕,11,0,0,0,2,0,1,5,0,0,1,0,0
신상용,0,0,0,0,1,0,0,0,0,1,0,0,0
이정선,7,1,0,0,1,0,1,4,0,0,1,0,0
이희영,3,0,0,0,2,0,1,4,0,0,3,0,0
최미연,2,2,2,1,1,0,0,6,2,1,0,0,0
이재영,11,1,2,0,1,0,0,0,0,5,0,0,0
김재형,0,2,3,0,3,0,0,0,1,5,0,0,0


In [225]:
df3 = df2.T
df3

직원성명,전현호,정성욱,박선녕,신상용,이정선,이희영,최미연,이재영,김재형
노란코끼리,3,1,11,0,7,3,2,11,0
어장촌생선구이,6,0,0,0,1,0,2,1,2
횡성목장,3,4,0,0,0,0,2,2,3
항도(港都),0,1,0,0,0,0,1,0,0
전주콩나루,1,2,2,1,1,2,1,1,3
옛날칼국수,0,1,0,0,0,0,0,0,0
칠리사이공,0,0,1,0,1,1,0,0,0
도나한우,0,6,5,0,4,4,6,0,0
북경(北京),0,1,0,0,0,0,2,0,1
명문식당,6,4,0,1,0,0,1,5,5


In [226]:
df3.loc['<식권 총계>'] = df3.sum(axis=0)
df3

직원성명,전현호,정성욱,박선녕,신상용,이정선,이희영,최미연,이재영,김재형
노란코끼리,3,1,11,0,7,3,2,11,0
어장촌생선구이,6,0,0,0,1,0,2,1,2
횡성목장,3,4,0,0,0,0,2,2,3
항도(港都),0,1,0,0,0,0,1,0,0
전주콩나루,1,2,2,1,1,2,1,1,3
옛날칼국수,0,1,0,0,0,0,0,0,0
칠리사이공,0,0,1,0,1,1,0,0,0
도나한우,0,6,5,0,4,4,6,0,0
북경(北京),0,1,0,0,0,0,2,0,1
명문식당,6,4,0,1,0,0,1,5,5


In [227]:
df3['<식당별 식권 개수>'] = df3.sum(axis=1)
df3

직원성명,전현호,정성욱,박선녕,신상용,이정선,이희영,최미연,이재영,김재형,<식당별 식권 개수>
노란코끼리,3,1,11,0,7,3,2,11,0,38
어장촌생선구이,6,0,0,0,1,0,2,1,2,12
횡성목장,3,4,0,0,0,0,2,2,3,14
항도(港都),0,1,0,0,0,0,1,0,0,2
전주콩나루,1,2,2,1,1,2,1,1,3,14
옛날칼국수,0,1,0,0,0,0,0,0,0,1
칠리사이공,0,0,1,0,1,1,0,0,0,3
도나한우,0,6,5,0,4,4,6,0,0,25
북경(北京),0,1,0,0,0,0,2,0,1,4
명문식당,6,4,0,1,0,0,1,5,5,22


In [228]:
df3.to_excel("12월 식권 사용내역.xlsx")

## 식권 처리 끝 !!